## Discover IPv6 and 6LoWPAN

In this excercice you will learn how you should communicate with IPV6 and wireless low power devices. The 6LoWPAN protocol has been developed to define the IPv6 adaptation and the way the IP datagrams will be transported over the IEEE802.15.4 radio links. You will deploy a private IPv6 network and test the connectivity between nodes. Moreover you will setup a public IPv6 network with a border router and verify that you can communicate with public servers. Finally with the monitoring feature you will configure a radio sniffer and analyse the traffic. With wireshark, a network protocol analyzer, you will inspect the packets from the different protocols layers with headers and payloads.

### IPv6 overview

Before going into the explanations of network configuration on the nodes, it is essential to have some notions about IPv6. Unlike an IPv4 address which is coded on 32 bits (ie 4 bytes) and uses a decimal notation (for example: 192.168.6.19), an IPv6 address is represented by a series of 128 bits (16 bytes), and is represented with a hexadecimal notation. For example, a public IPv6 address (so-called "global" unicast address, that is to say routable) on IoT-LAB Grenoble site can have the full representation:

`2001:0660:5307:30ff:0000:0000:0000:0001` can be shortened to `2001:660:5307:30ff::1` (a series of 0 contiguous is replaced only once by ::)

This 128-bit series is often divided into 2 parts:

* the least significant 64 bits correspond to the address of the host, `::1` in the previous example. Generally, they are constructed from the MAC address of the host to guarantee the uniqueness of an IPv6 address in a subnet (since the physical address isrmally);

* the most significant 64 bits correspond to the network address, `2001:660:5307:30ff::/64` in the previous example. They contain in particular the "prefix" used for routing IPv6 packets and use, as in IPv4, the CIDR notation: <prefix> / <bit length>. This 64-bit block is divided into a first part containing up to 48 bits and designating the "site prefix",  (`2001:660:5307`) the rest of the bits identifying the subnet (`30ff`). A prefix always contains 64 bits.

Some prefixes are reserved for very specific uses:

* `fe80::/10` is used for unicast addresses called "link-local". This type of address only allows 2 network nodes to communicate if they share a direct physical link. This was the case with two nodes with wireless interface (802.15.4),

* `fd00::/8` is intended for "Unique Local Address" addresses. These addresses correspond to private addresses in IPv4 and are not routable.


### Launch an experiment

1. Choose your site (grenoble|lille|saclay|strasbourg|...):

In [6]:
%env SITE = grenoble

env: SITE=grenoble


In [27]:
!iotlab-experiment submit -d 120 -l 2,archi=m3:at86rf231+site=$SITE

{
    "id": 193154
}


2. Wait for the experiment to be in the Running state:

In [28]:
!iotlab-experiment wait

Waiting that experiment 193154 gets in state Running
"Running"


3. Check the resources allocated to the experiment

In [1]:
!iotlab-experiment get -ri

{
    "items": [
        {
            "grenoble": {
                "m3": "100-101"
            }
        }
    ]
}


### Communication between two nodes

With the Contiki-NG ``hello world`` example and shell enabled you can use the ping6 command and test the IPv6 connectivity between two nodes

Let's define an environment variable to use the IoT-LAB Contiki-NG port and compile the example


In [2]:
%env ARCH_PATH=../../../iot-lab-contiki-ng/arch

env: ARCH_PATH=../../../iot-lab-contiki-ng/arch


In [3]:
!TARGET=iotlab BOARD=m3 make -C hello-world

make: Entering directory '/home/user/iot-lab-training/contiki-ng/networking/ipv6/hello-world'
  CP        build/iotlab/m3/hello-world.iotlab --> hello-world.iotlab
make: Leaving directory '/home/user/iot-lab-training/contiki-ng/networking/ipv6/hello-world'


The binary firmware ``hello-world.iotlab`` is available in the ``hello-world`` directory.

You can flash it on the two nodes with the following command

In [26]:
!iotlab-node --update hello-world/hello-world.iotlab

Killed


At this stage use `File > New > Terminal` two times, one for each node, and connect to the IoT-LAB SSH frontend to have access of the node's serial link. 
On the first shell choose one node and connect to the serial port with ``nc`` command on the port 20000. Next print the shell usage with the ``help`` command and use ``ip-addr`` command to show IPV6 "link-local" address.

On the other node try to ping the IPv6 address and test the connectivity. You can note that we used the IPv6 "link-local" address of the node. We don't have a global IPV6 address and we can only communicate because the two nodes are in the same radio neighborhood with the same physical link (802.15.4 and channel 26)

### Communication via a Border Router (BR)

In order for a node to communicate in IPv6 with a host on the Internet, it needs a “global” unicast address. To do this, a Border Router node  must be added to the network, which will be responsible for propagating the IPv6 global prefix to the other nodes. We speak here of border router, because it is on the border between an 802.15.4 network and a classic Ethernet network. For this choose one node of your experiment which takes the role of BR. 

1. Compile and flash border router firmware

In [3]:
!TARGET=iotlab BOARD=m3 make -C rpl-border-router

make: Entering directory '/home/user/iot-lab-training/contiki-ng/networking/ipv6/hello-world'
  CP        build/iotlab/m3/hello-world.iotlab --> hello-world.iotlab
make: Leaving directory '/home/user/iot-lab-training/contiki-ng/networking/ipv6/hello-world'


Choose one node which play the role of BR and flash the firmware only on this node as below

In [26]:
!iotlab-node --update rpl-border-router/border-router.iotlab -l grenoble,m3,<id1>

Killed


use `File > New > Terminal` and connect to the IoT-LAB SSH frontend to have access of the BR node's serial link.

We need to create a network interface and choose a public IPv6 prefix available on the frontend. As it's a shared environment you must check before.
On Grenoble site you have 128 IPv6 public prefix available:

<code>2001:660:5307:3100 -> 2001:660:5307:317f</code>

* Visualize which prefix are already used

Launch the following command with IPv6 prefix and the good node's id for the border router:

This runs the program tunslip6, which bridges the Contiki-NG border router node to the SSH frontend server via a tun interface. Tunslip uses SLIP (Serial Line Internet Protocol) to encapsulate and pass IP traffic to and from the other side of the serial line. The RPL border router, running on the node, has an auto-configured address which you can find out from the above logs. Your host is assigned address ``2001:660:5307:3100::b080``.

To access nodes inside the RPL network, we first need to find out their global IPv6 address. To help with this, the RPL border router runs an HTTP server. You can request it:

The page contains a list of routes with the addresses of all nodes in the network.

You can also check that on the other node shell with the `ip-addr` command you visualize the global ipv6 address in addition to the ``link-local`` address

The BR node starts advertising the network via RPL DIO (DODAG Information Object) messages. Nodes that receive a DIO message will join the RPL network. Nodes can also search for available networks by sending periodic DIS (DODAG Information Solicitation) messages. You can find more informations about RPL implementation in the [Contiki-NG documentation](https://github.com/contiki-ng/contiki-ng/wiki/Documentation:-RPL)

The last step is to test the IPv6 connectivity with global internet and the node. Try to ping a public Google server

### Radio sniffer

One of the key feature of IoT-LAB is providing automatic monitoring on energy consumption and radio activity, thanks to a Control Node, associated to the experiment node. You do not have to bring a firmware for this Control Node, but just specify its configuration through what we called a 'profile'. Each user can manage his collection of profiles. Here you will create one.

Create a sniffer profile on the default Contiki-NG radio channel

use `File > New > Terminal` and launch the command

In [14]:
!iotlab-profile addm3 -n sniff_26 -sniffer -channels 26

{
    "created": "sniff_26"
}


After you have just to update the profile configuration of your node

In [14]:
!iotlab-node --update-profile sniff_26 -l grenoble,m3,<id>

{
    "created": "sniff_26"
}


Connect to the SSH frontend and launch the sniffer_aggregator command. This command aggregates all the nodes sniffer links (TCP socket on port 30000). By default t it encapsulates packet as ZigBee Encapsulation Protocol (ZEP). With the -r option 802.15.4 payload are extracted and saved in a 802.15.4_link_layer pcap file directly usable in wireshark network traffic analyser.

Go to the shell node and ping6 Google server

On the sniffer aggregator shell you must see that you have captured packets. Stop it with Ctr^C shortcut when you have at least 6 packets captured. Edit the pcap file and analyse the traffic.


From the frame1 (first packet) you can see the IEEE 802.15.4 data, 6LoWPAN and Internet Protocol version 6 layer.

* In the 802.15.4 section you can see the mac address of the source and destination packet. You can retrieve the mac address of your node which executes the ping6 address in the Src field.

* In the 6LoWPAN section you can see the IPV6 address of the source and destination packet.

* In the IPv6 section you can see the ICMP message which corresponds to a Echo ping request (ping6 command)

From the frame2 you should see the ICMP message from the Google server which corresponds to Echo ping reply.

In the next frames you can discover the Neighbor Discovery Protocol traffic between the border router and the node. You should see Neighbor Solicitation and Advertisement message exchanged  between the border router and the node.
